In [1]:
import requests
import time
import csv
from pyquery import PyQuery as pq

In [2]:
class mySpider:
    
    def __init__(self):
        self.base_url = 'https://maoyan.com/board/4?offset={}'
        self.headers  = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
        }
    
    # 创建一个csv文件，建立headers
    def create_file(self):
        # python 读写csv文件（创建，追加，覆盖） https://blog.csdn.net/lwgkzl/article/details/82147474
        with open('./raw_data.csv', 'w') as f:
            csv_write = csv.writer(f)
            csv_head = ["title","actor","releasetime","stars","style","location","film_duration"]
            csv_write.writerow(csv_head)
     
    # 得到完整的url地址
    def get_url(self,num):
        desire_url = self.base_url.format(num)
        self.get_page(desire_url)
    
    # 根据url地址，用requests库获取相应页面的内容 并传入分析页面的函数 
    def get_page(self,url):
            response = requests.get(url,self.headers)
            if response.status_code == 200:
                self.parse_page(response.content.decode('utf-8'))
                # 每次睡眠3秒，防止被网页发现爬虫行为
                time.sleep(3)
            else:
                print('Failed to load data from desired url')
    
    # 利用pyquery从获取的页面内容里提取我们想要的相关信息 并存入到本地的CSV文件内
    def parse_page(self,html):
        data  = pq(html) 
        list1 = data('.board-wrapper dd').items()
        for lis in list1:
            title = lis('p.name a').text()
            actor = lis('p.star').text().split('主演：')[1]
            releasetime = lis('p.releasetime').text().split('上映时间：')[1][0:10]
            star  = float(lis('p.score i:nth-child(1)').text() + lis('p.score i:nth-child(2)').text())
            final_url = 'https://maoyan.com' + lis('.movie-item-info .name a').attr('href')
            style,location,duration = self.more_info(final_url)
            
            # 写入csv文件
            with open('./raw_data.csv', 'a', newline='', encoding='utf-8') as f:
                csv_write = csv.writer(f)
                data_row = [title,actor,releasetime,star,style,location,duration]
                csv_write.writerow(data_row)
            
    def more_info(self,url):
        data  = pq(url)
        style = data('li.ellipsis:nth-child(1)').text().strip()
        location = data('li.ellipsis:nth-child(2)').text().split('/')[0].strip()
        film_duration = data('li.ellipsis:nth-child(2)').text().split('/')[1].strip()
        return style, location, film_duration
    
    # 简历类后，通过单独调用这个函数实现整个爬虫的功能
    def execution(self):
        self.create_file()
        for i in range(0,100,10):
            self.get_url(i)

In [3]:
if __name__ == '__main__':
    crawler = mySpider()
    crawler.execution()